In [ ]:
!pip install -U geemap
!pip install fiona geojson geopandas rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 53.9 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
geemap.update_package()
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import fiona
import geojson
import json
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from scipy.ndimage import generic_filter

Unzipping geemap-master.zip ...
Data downloaded to: /root/Downloads/geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-jayantakurp17")

In [ ]:
Map = geemap.Map(center=(22.513989463375896, 89.59764297714862), zoom=9)
Map

Map(center=[22.513989463375896, 89.59764297714862], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
m = Map.draw_last_feature
aoi = ee.FeatureCollection(m).geometry()
Map.addLayer(aoi, {}, 'Area of Interest')

In [ ]:
sImage = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
                        .filterBounds(aoi) \
                        .filterDate('2020-01-01', '2020-03-30') \
                        .filterMetadata('CLOUD_COVER', 'less_than', 0.1)

sImage8 = sImage.median().clip(aoi)
Map.addLayer(sImage8, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Landsat-8 Image')

**NDVI**

In [ ]:
  # Function to calculate NDVI
def calculate_ndvi(image):
      nir = image.select('B5')
      red = image.select('B4')
      ndvi = image.expression(
          '(NIR-RED)/(NIR+RED)',
          {
              'NIR': nir,
              'RED': red
          }
      ).rename('NDVI')
      return ndvi

**Wetness**

In [ ]:
def calculate_wet(image):
  blue = image.select('B2')
  green = image.select('B3')
  red = image.select('B4')
  nir = image.select('B5')
  swir1 = image.select('B6')
  swir2 = image.select('B7')
  wetness = image.expression(
      '(0.1511*BLUE) +(0.1973*GREEN)+(0.3283*RED)+(0.3407*NIR)-(0.7117*SWIR1)-(0.4559*SWIR2)'
      , {
          'BLUE': blue,
          'GREEN': green,
          'RED': red,
          'NIR': nir,
          'SWIR1': swir1,
          'SWIR2': swir2
      }).rename('Wetness')
  return wetness


**NDBSI**

In [ ]:
def calculate_ndbsi(image):
  blue = image.select('B2')
  green = image.select('B3')
  red = image.select('B4')
  nir = image.select('B5')
  swir1= image.select('B6')
  swir2 = image.select('B7')
  si= image.expression(
      '((SWIR1+RED)-(NIR+BLUE))/((SWIR1+RED)+(NIR+BLUE))',
       {
          'NIR': nir,
          'RED': red,
          'GREEN': green,
          'SWIR1': swir1,
          'SWIR2': swir2,
          'BLUE': blue
      }).rename('SI')
  ibi = image.expression(
      '(2 * SWIR1 / (SWIR1 + NIR)) - ((NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1)))/(2 * SWIR1 / (SWIR1 + NIR)) + ((NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1)))', {
          'NIR': nir,
          'RED': red,
          'GREEN': green,
          'SWIR1': swir1,
          'SWIR2': swir2
      }).rename('IBI')
  ndbsi = image.expression(
      '(IBI+SI)/2', {
          'IBI': ibi,
          'SI': si
      }).rename('NDBSI')
  return ndbsi

**LST**

In [ ]:
# Function to calculate LST
def calculate_lst(image):
  # Step 1: Calculate NDVI
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')

  # Step 2: Calculate minimum and maximum NDVI in the AOI
  ndviMin = ee.Number(ndvi.reduceRegion(
          reducer=ee.Reducer.min(),
          geometry=aoi,
          scale=30,
          maxPixels=1e9
      ).values().get(0))

  ndviMax = ee.Number(ndvi.reduceRegion(
          reducer=ee.Reducer.max(),
          geometry=aoi,
          scale=30,
          maxPixels=1e9
      ).values().get(0))

      # Step 3: Calculate Fractional Vegetation (FV)
  fv = ndvi.subtract(ndviMin) \
              .divide(ndviMax.subtract(ndviMin)) \
              .pow(ee.Number(2)) \
              .rename('FV')

      # Step 4: Calculate Emissivity (EM)
  em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

      # Step 5: Select Thermal Band (Band 10)
  thermal = image.select('B10').rename('thermal')

      # Step 6: Calculate Land Surface Temperature (LST)
  lst = thermal.expression(
          '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
              'TB': thermal.select('thermal'),  # Brightness temperature in Kelvin
              'em': em  # Emissivity
          }).rename('LST')
  return lst

**Calculation**

In [ ]:
ndvi = calculate_ndvi(sImage8)
wetness = calculate_wet(sImage8)
ndbsi = calculate_ndbsi(sImage8)
lst = calculate_lst(sImage8)

In [ ]:
def export_image(image, image_name):
      # Create and start the export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{image_name}",
            folder="GEE_Exports",
            fileNamePrefix=image_name,
            region=aoi,
            fileFormat='GeoTIFF',
            scale=30,
            maxPixels=1e13
        )
        task.start()
        print(f"Export task for '{image_name}' started. Check Google Drive for output.")

In [ ]:
export_image(ndvi, "ndvi_20")
export_image(wetness, "wetness_20")


Export task for 'ndvi_20' started. Check Google Drive for output.
Export task for 'wetness_20' started. Check Google Drive for output.
Export task for 'ndbsi_20' started. Check Google Drive for output.
Export task for 'lst_20' started. Check Google Drive for output.


In [ ]:
export_image(ndbsi, "ndbsi_20")
export_image(lst, "lst_20")

Export task for 'ndbsi_20' started. Check Google Drive for output.
Export task for 'lst_20' started. Check Google Drive for output.
